# Q2: Mô hình Dự đoán Khoảng Giá Nhà (Prediction Intervals)

## Mục tiêu
- Xây dựng mô hình dự đoán khoảng giá nhà (lower bound, upper bound)
- So sánh các phương pháp: Quantile Regression, LightGBM, Ensemble
- Tối ưu hóa Mean Interval Width trên Kaggle

In [ ]:
# Import thư viện
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import QuantileRegressor, LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import lightgbm as lgb
import xgboost as xgb
from scipy import stats
import joblib
import warnings
warnings.filterwarnings('ignore')

# Cấu hình
plt.style.use('seaborn-v0_8')
np.random.seed(42)

In [ ]:
# Load dữ liệu đã xử lý từ Q1
try:
    X_train = pd.read_csv('processed_data/X_train_processed.csv')
    X_test = pd.read_csv('processed_data/X_test_processed.csv')
    y_train = pd.read_csv('processed_data/y_train.csv')['SalePrice']
    scaler = joblib.load('processed_data/scaler.pkl')
    
    print(f"✅ Loaded processed data successfully")
    print(f"X_train shape: {X_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    print(f"y_train shape: {y_train.shape}")
    
except FileNotFoundError:
    print("❌ Processed data not found. Please run Q1 notebook first.")
    # Fallback: load raw data
    print("Loading raw data as fallback...")
    train_df = pd.read_csv('train.csv')
    test_df = pd.read_csv('test.csv')
    
    # Basic preprocessing
    numeric_cols = train_df.select_dtypes(include=[np.number]).columns
    X_train = train_df[numeric_cols].drop('SalePrice', axis=1).fillna(0)
    y_train = train_df['SalePrice']
    X_test = test_df[numeric_cols].fillna(0)

## 1. Prediction Intervals Framework

In [ ]:
class PredictionIntervalModel:
    def __init__(self, alpha=0.1, random_state=42):
        """
        alpha: significance level (0.1 for 90% confidence interval)
        """
        self.alpha = alpha
        self.lower_quantile = alpha / 2
        self.upper_quantile = 1 - alpha / 2
        self.random_state = random_state
        self.models = {}
        
    def calculate_interval_width(self, lower_pred, upper_pred):
        """Tính độ rộng interval trung bình"""
        return np.mean(upper_pred - lower_pred)
    
    def calculate_coverage_rate(self, y_true, lower_pred, upper_pred):
        """Tính tỷ lệ coverage"""
        covered = (y_true >= lower_pred) & (y_true <= upper_pred)
        return np.mean(covered)
    
    def evaluate_intervals(self, y_true, lower_pred, upper_pred):
        """Đánh giá chất lượng prediction intervals"""
        width = self.calculate_interval_width(lower_pred, upper_pred)
        coverage = self.calculate_coverage_rate(y_true, lower_pred, upper_pred)
        
        return {
            'mean_interval_width': width,
            'coverage_rate': coverage,
            'target_coverage': 1 - self.alpha
        }

# Khởi tạo model
pi_model = PredictionIntervalModel(alpha=0.1)  # 90% confidence interval
print(f"Prediction Interval Model initialized")
print(f"Target coverage: {1-pi_model.alpha:.1%}")
print(f"Lower quantile: {pi_model.lower_quantile:.3f}")
print(f"Upper quantile: {pi_model.upper_quantile:.3f}")

## 2. Phương pháp 1: Quantile Regression

In [ ]:
def quantile_regression_approach(X_train, y_train, X_test, alpha=0.1):
    """Phương pháp Quantile Regression"""
    lower_quantile = alpha / 2
    upper_quantile = 1 - alpha / 2
    
    print(f"🔧 Training Quantile Regression models...")
    
    # Model cho quantile dưới
    print(f"   Training lower quantile model ({lower_quantile:.3f})...")
    lower_model = QuantileRegressor(
        quantile=lower_quantile, 
        alpha=0.01, 
        solver='highs'
    )
    lower_model.fit(X_train, y_train)
    
    # Model cho quantile trên
    print(f"   Training upper quantile model ({upper_quantile:.3f})...")
    upper_model = QuantileRegressor(
        quantile=upper_quantile, 
        alpha=0.01, 
        solver='highs'
    )
    upper_model.fit(X_train, y_train)
    
    # Dự đoán
    print(f"   Making predictions...")
    lower_pred = lower_model.predict(X_test)
    upper_pred = upper_model.predict(X_test)
    
    # Đảm bảo lower <= upper
    lower_pred = np.minimum(lower_pred, upper_pred)
    upper_pred = np.maximum(lower_pred, upper_pred)
    
    return lower_pred, upper_pred, {'lower_model': lower_model, 'upper_model': upper_model}

# Chia train/validation để đánh giá
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

# Huấn luyện Quantile Regression
qr_lower_val, qr_upper_val, qr_models = quantile_regression_approach(
    X_train_split, y_train_split, X_val_split
)

# Đánh giá
qr_metrics = pi_model.evaluate_intervals(y_val_split, qr_lower_val, qr_upper_val)
print(f"\n📊 Quantile Regression Results:")
print(f"   Mean Interval Width: {qr_metrics['mean_interval_width']:,.0f}")
print(f"   Coverage Rate: {qr_metrics['coverage_rate']:.3f}")
print(f"   Target Coverage: {qr_metrics['target_coverage']:.3f}")

## 3. Phương pháp 2: LightGBM Quantile

In [ ]:
def lightgbm_quantile_approach(X_train, y_train, X_test, alpha=0.1):
    """Phương pháp LightGBM với quantile objective"""
    lower_quantile = alpha / 2
    upper_quantile = 1 - alpha / 2
    
    print(f"🔧 Training LightGBM Quantile models...")
    
    # Tham số chung
    base_params = {
        'objective': 'quantile',
        'metric': 'quantile',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'learning_rate': 0.05,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': -1,
        'random_state': 42
    }
    
    # Chuẩn bị dữ liệu
    train_data = lgb.Dataset(X_train, label=y_train)
    
    # Model cho quantile dưới
    print(f"   Training lower quantile model ({lower_quantile:.3f})...")
    lower_params = base_params.copy()
    lower_params['alpha'] = lower_quantile
    
    lower_model = lgb.train(
        lower_params, 
        train_data, 
        num_boost_round=1000,
        callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)]
    )
    
    # Model cho quantile trên
    print(f"   Training upper quantile model ({upper_quantile:.3f})...")
    upper_params = base_params.copy()
    upper_params['alpha'] = upper_quantile
    
    upper_model = lgb.train(
        upper_params, 
        train_data, 
        num_boost_round=1000,
        callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)]
    )
    
    # Dự đoán
    print(f"   Making predictions...")
    lower_pred = lower_model.predict(X_test, num_iteration=lower_model.best_iteration)
    upper_pred = upper_model.predict(X_test, num_iteration=upper_model.best_iteration)
    
    # Đảm bảo lower <= upper
    lower_pred = np.minimum(lower_pred, upper_pred)
    upper_pred = np.maximum(lower_pred, upper_pred)
    
    return lower_pred, upper_pred, {'lower_model': lower_model, 'upper_model': upper_model}

# Huấn luyện LightGBM
lgb_lower_val, lgb_upper_val, lgb_models = lightgbm_quantile_approach(
    X_train_split, y_train_split, X_val_split
)

# Đánh giá
lgb_metrics = pi_model.evaluate_intervals(y_val_split, lgb_lower_val, lgb_upper_val)
print(f"\n📊 LightGBM Quantile Results:")
print(f"   Mean Interval Width: {lgb_metrics['mean_interval_width']:,.0f}")
print(f"   Coverage Rate: {lgb_metrics['coverage_rate']:.3f}")
print(f"   Target Coverage: {lgb_metrics['target_coverage']:.3f}")

## 4. Phương pháp 3: Ensemble với Uncertainty Estimation

In [ ]:
def ensemble_uncertainty_approach(X_train, y_train, X_test, alpha=0.1):
    """Phương pháp Ensemble với ước tính uncertainty"""
    print(f"🔧 Training Ensemble models...")
    
    models = []
    predictions = []
    
    # 1. Random Forest
    print(f"   Training Random Forest...")
    rf = RandomForestRegressor(
        n_estimators=100, 
        max_depth=10,
        random_state=42,
        n_jobs=-1
    )
    rf.fit(X_train, y_train)
    rf_pred = rf.predict(X_test)
    models.append(('RandomForest', rf))
    predictions.append(rf_pred)
    
    # 2. Gradient Boosting
    print(f"   Training Gradient Boosting...")
    gb = GradientBoostingRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=6,
        random_state=42
    )
    gb.fit(X_train, y_train)
    gb_pred = gb.predict(X_test)
    models.append(('GradientBoosting', gb))
    predictions.append(gb_pred)
    
    # 3. XGBoost
    print(f"   Training XGBoost...")
    xgb_model = xgb.XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=6,
        random_state=42
    )
    xgb_model.fit(X_train, y_train)
    xgb_pred = xgb_model.predict(X_test)
    models.append(('XGBoost', xgb_model))
    predictions.append(xgb_pred)
    
    # Tính ensemble prediction
    print(f"   Computing ensemble predictions...")
    predictions = np.array(predictions)
    ensemble_pred = np.mean(predictions, axis=0)
    
    # Ước tính uncertainty từ variance của predictions
    pred_std = np.std(predictions, axis=0)
    
    # Tính prediction intervals
    z_score = stats.norm.ppf(1 - alpha/2)  # for confidence interval
    
    lower_pred = ensemble_pred - z_score * pred_std
    upper_pred = ensemble_pred + z_score * pred_std
    
    return lower_pred, upper_pred, {
        'models': models, 
        'ensemble_pred': ensemble_pred,
        'pred_std': pred_std
    }

# Huấn luyện Ensemble
ens_lower_val, ens_upper_val, ens_models = ensemble_uncertainty_approach(
    X_train_split, y_train_split, X_val_split
)

# Đánh giá
ens_metrics = pi_model.evaluate_intervals(y_val_split, ens_lower_val, ens_upper_val)
print(f"\n📊 Ensemble Uncertainty Results:")
print(f"   Mean Interval Width: {ens_metrics['mean_interval_width']:,.0f}")
print(f"   Coverage Rate: {ens_metrics['coverage_rate']:.3f}")
print(f"   Target Coverage: {ens_metrics['target_coverage']:.3f}")

## 5. So sánh các Phương pháp

In [ ]:
# So sánh kết quả
def compare_methods():
    results = {
        'Method': ['Quantile Regression', 'LightGBM Quantile', 'Ensemble Uncertainty'],
        'Mean_Interval_Width': [
            qr_metrics['mean_interval_width'],
            lgb_metrics['mean_interval_width'],
            ens_metrics['mean_interval_width']
        ],
        'Coverage_Rate': [
            qr_metrics['coverage_rate'],
            lgb_metrics['coverage_rate'],
            ens_metrics['coverage_rate']
        ],
        'Coverage_Gap': [
            abs(qr_metrics['coverage_rate'] - qr_metrics['target_coverage']),
            abs(lgb_metrics['coverage_rate'] - lgb_metrics['target_coverage']),
            abs(ens_metrics['coverage_rate'] - ens_metrics['target_coverage'])
        ]
    }
    
    comparison_df = pd.DataFrame(results)
    comparison_df['Rank_Width'] = comparison_df['Mean_Interval_Width'].rank()
    comparison_df['Rank_Coverage'] = comparison_df['Coverage_Gap'].rank()
    comparison_df['Overall_Score'] = comparison_df['Rank_Width'] + comparison_df['Rank_Coverage']
    
    print("="*80)
    print("                    SO SÁNH CÁC PHƯƠNG PHÁP")
    print("="*80)
    print(comparison_df.round(3))
    
    # Tìm phương pháp tốt nhất
    best_method_idx = comparison_df['Overall_Score'].idxmin()
    best_method = comparison_df.loc[best_method_idx, 'Method']
    
    print(f"\n🏆 PHƯƠNG PHÁP TỐT NHẤT: {best_method}")
    print(f"   • Mean Interval Width: {comparison_df.loc[best_method_idx, 'Mean_Interval_Width']:,.0f}")
    print(f"   • Coverage Rate: {comparison_df.loc[best_method_idx, 'Coverage_Rate']:.3f}")
    
    return comparison_df, best_method

comparison_results, best_method = compare_methods()

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Interval Width Comparison
methods = comparison_results['Method']
widths = comparison_results['Mean_Interval_Width']
colors = ['skyblue', 'lightcoral', 'lightgreen']

axes[0,0].bar(methods, widths, color=colors)
axes[0,0].set_title('Mean Interval Width Comparison', fontweight='bold')
axes[0,0].set_ylabel('Mean Interval Width ($)')
axes[0,0].tick_params(axis='x', rotation=45)
for i, v in enumerate(widths):
    axes[0,0].text(i, v + max(widths)*0.01, f'{v:,.0f}', ha='center', va='bottom')

# 2. Coverage Rate Comparison
coverage_rates = comparison_results['Coverage_Rate']
target_coverage = 0.9

axes[0,1].bar(methods, coverage_rates, color=colors)
axes[0,1].axhline(y=target_coverage, color='red', linestyle='--', label=f'Target: {target_coverage:.1%}')
axes[0,1].set_title('Coverage Rate Comparison', fontweight='bold')
axes[0,1].set_ylabel('Coverage Rate')
axes[0,1].tick_params(axis='x', rotation=45)
axes[0,1].legend()
for i, v in enumerate(coverage_rates):
    axes[0,1].text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')

# 3. Prediction Intervals Visualization (sample)
sample_size = 50
sample_idx = np.random.choice(len(y_val_split), sample_size, replace=False)
x_pos = np.arange(sample_size)

axes[1,0].scatter(x_pos, y_val_split.iloc[sample_idx], color='red', label='True Values', s=30, alpha=0.7)
axes[1,0].fill_between(x_pos, lgb_lower_val[sample_idx], lgb_upper_val[sample_idx], 
                       alpha=0.3, color='blue', label='LightGBM Intervals')
axes[1,0].set_title('Sample Prediction Intervals (LightGBM)', fontweight='bold')
axes[1,0].set_xlabel('Sample Index')
axes[1,0].set_ylabel('Sale Price ($)')
axes[1,0].legend()

# 4. Residuals Analysis
lgb_center = (lgb_lower_val + lgb_upper_val) / 2
residuals = y_val_split - lgb_center

axes[1,1].scatter(lgb_center, residuals, alpha=0.6)
axes[1,1].axhline(y=0, color='red', linestyle='--')
axes[1,1].set_title('Residuals vs Predicted (LightGBM)', fontweight='bold')
axes[1,1].set_xlabel('Predicted Center')
axes[1,1].set_ylabel('Residuals')

plt.tight_layout()
plt.show()

## 6. Cross-Validation Evaluation

In [ ]:
# Cross-validation cho phương pháp tốt nhất
def cross_validate_prediction_intervals(X, y, method='lightgbm', cv_folds=5):
    """Cross-validation cho prediction intervals"""
    kf = KFold(n_splits=cv_folds, shuffle=True, random_state=42)
    
    cv_widths = []
    cv_coverages = []
    
    print(f"🔄 Running {cv_folds}-fold Cross-Validation for {method}...")
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        print(f"   Fold {fold + 1}/{cv_folds}...")
        
        X_train_cv = X.iloc[train_idx]
        X_val_cv = X.iloc[val_idx]
        y_train_cv = y.iloc[train_idx]
        y_val_cv = y.iloc[val_idx]
        
        if method == 'lightgbm':
            lower_pred, upper_pred, _ = lightgbm_quantile_approach(
                X_train_cv, y_train_cv, X_val_cv
            )
        elif method == 'quantile':
            lower_pred, upper_pred, _ = quantile_regression_approach(
                X_train_cv, y_train_cv, X_val_cv
            )
        elif method == 'ensemble':
            lower_pred, upper_pred, _ = ensemble_uncertainty_approach(
                X_train_cv, y_train_cv, X_val_cv
            )
        
        # Đánh giá fold này
        width = np.mean(upper_pred - lower_pred)
        coverage = np.mean((y_val_cv >= lower_pred) & (y_val_cv <= upper_pred))
        
        cv_widths.append(width)
        cv_coverages.append(coverage)
    
    # Tổng hợp kết quả
    cv_results = {
        'mean_width': np.mean(cv_widths),
        'std_width': np.std(cv_widths),
        'mean_coverage': np.mean(cv_coverages),
        'std_coverage': np.std(cv_coverages),
        'fold_widths': cv_widths,
        'fold_coverages': cv_coverages
    }
    
    print(f"\n📊 Cross-Validation Results ({method}):")
    print(f"   Mean Interval Width: {cv_results['mean_width']:,.0f} ± {cv_results['std_width']:,.0f}")
    print(f"   Mean Coverage Rate: {cv_results['mean_coverage']:.3f} ± {cv_results['std_coverage']:.3f}")
    
    return cv_results

# Chạy CV cho phương pháp tốt nhất
if 'LightGBM' in best_method:
    cv_results = cross_validate_prediction_intervals(X_train, y_train, 'lightgbm')
elif 'Quantile' in best_method:
    cv_results = cross_validate_prediction_intervals(X_train, y_train, 'quantile')
else:
    cv_results = cross_validate_prediction_intervals(X_train, y_train, 'ensemble')

## 7. Final Model Training và Prediction

In [ ]:
# Huấn luyện mô hình cuối cùng trên toàn bộ training data
print(f"🚀 Training final model on full training data...")

if 'LightGBM' in best_method:
    final_lower, final_upper, final_models = lightgbm_quantile_approach(
        X_train, y_train, X_test
    )
    model_type = 'lightgbm'
elif 'Quantile' in best_method:
    final_lower, final_upper, final_models = quantile_regression_approach(
        X_train, y_train, X_test
    )
    model_type = 'quantile'
else:
    final_lower, final_upper, final_models = ensemble_uncertainty_approach(
        X_train, y_train, X_test
    )
    model_type = 'ensemble'

print(f"✅ Final model training completed!")
print(f"   Test predictions shape: {final_lower.shape}")
print(f"   Mean interval width: {np.mean(final_upper - final_lower):,.0f}")

## 8. Tạo Submission File

In [ ]:
# Tạo submission file cho Kaggle
def create_submission(lower_pred, upper_pred, test_ids=None):
    """Tạo file submission cho Kaggle"""
    
    # Nếu không có test_ids, tạo ID giả
    if test_ids is None:
        test_ids = range(len(lower_pred))
    
    submission = pd.DataFrame({
        'Id': test_ids,
        'lower': lower_pred,
        'upper': upper_pred
    })
    
    # Đảm bảo lower <= upper
    submission['lower'] = np.minimum(submission['lower'], submission['upper'])
    submission['upper'] = np.maximum(submission['lower'], submission['upper'])
    
    # Làm tròn
    submission['lower'] = submission['lower'].round(2)
    submission['upper'] = submission['upper'].round(2)
    
    return submission

# Tạo submission
try:
    # Thử load test IDs từ file gốc
    test_df_original = pd.read_csv('test.csv')
    test_ids = test_df_original['Id'].values if 'Id' in test_df_original.columns else None
except:
    test_ids = None

submission = create_submission(final_lower, final_upper, test_ids)

# Lưu submission
submission_filename = f'submission_{model_type}_{pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")}.csv'
submission.to_csv(submission_filename, index=False)

print(f"📄 Submission file created: {submission_filename}")
print(f"   Shape: {submission.shape}")
print(f"\nFirst 5 rows:")
print(submission.head())

print(f"\n📊 Submission Statistics:")
print(f"   Mean lower bound: ${submission['lower'].mean():,.0f}")
print(f"   Mean upper bound: ${submission['upper'].mean():,.0f}")
print(f"   Mean interval width: ${(submission['upper'] - submission['lower']).mean():,.0f}")
print(f"   Min interval width: ${(submission['upper'] - submission['lower']).min():,.0f}")
print(f"   Max interval width: ${(submission['upper'] - submission['lower']).max():,.0f}")

## 9. Model Interpretation và Feature Importance

In [ ]:
# Feature importance analysis
def analyze_feature_importance(models, model_type, feature_names):
    """Phân tích feature importance"""
    
    if model_type == 'lightgbm':
        # LightGBM feature importance
        lower_importance = models['lower_model'].feature_importance(importance_type='gain')
        upper_importance = models['upper_model'].feature_importance(importance_type='gain')
        
        # Trung bình của lower và upper
        avg_importance = (lower_importance + upper_importance) / 2
        
        feature_imp = pd.DataFrame({
            'feature': feature_names,
            'importance': avg_importance
        }).sort_values('importance', ascending=False)
        
    elif model_type == 'quantile':
        # Quantile regression coefficients
        lower_coef = np.abs(models['lower_model'].coef_)
        upper_coef = np.abs(models['upper_model'].coef_)
        
        avg_coef = (lower_coef + upper_coef) / 2
        
        feature_imp = pd.DataFrame({
            'feature': feature_names,
            'importance': avg_coef
        }).sort_values('importance', ascending=False)
        
    elif model_type == 'ensemble':
        # Ensemble feature importance
        importances = []
        for name, model in models['models']:
            if hasattr(model, 'feature_importances_'):
                importances.append(model.feature_importances_)
        
        if importances:
            avg_importance = np.mean(importances, axis=0)
            feature_imp = pd.DataFrame({
                'feature': feature_names,
                'importance': avg_importance
            }).sort_values('importance', ascending=False)
        else:
            return None
    
    return feature_imp

# Phân tích feature importance
feature_names = X_train.columns.tolist()
feature_importance = analyze_feature_importance(final_models, model_type, feature_names)

if feature_importance is not None:
    print("🔍 TOP 15 FEATURES QUAN TRỌNG NHẤT:")
    print(feature_importance.head(15))
    
    # Visualize feature importance
    plt.figure(figsize=(12, 8))
    top_features = feature_importance.head(20)
    
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Feature Importance')
    plt.title(f'Top 20 Feature Importance ({model_type.upper()})', fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
else:
    print("❌ Could not extract feature importance for this model type")

## 10. Lưu Models và Kết quả

In [ ]:
# Lưu models và kết quả
import os
os.makedirs('models', exist_ok=True)
os.makedirs('results', exist_ok=True)

# Lưu models
if model_type == 'lightgbm':
    final_models['lower_model'].save_model('models/lgb_lower_model.txt')
    final_models['upper_model'].save_model('models/lgb_upper_model.txt')
else:
    joblib.dump(final_models, f'models/{model_type}_models.pkl')

# Lưu kết quả
results_summary = {
    'best_method': best_method,
    'model_type': model_type,
    'cv_results': cv_results,
    'comparison_results': comparison_results.to_dict(),
    'submission_stats': {
        'mean_interval_width': float((submission['upper'] - submission['lower']).mean()),
        'min_interval_width': float((submission['upper'] - submission['lower']).min()),
        'max_interval_width': float((submission['upper'] - submission['lower']).max())
    }
}

import json
with open('results/model_results.json', 'w') as f:
    json.dump(results_summary, f, indent=2)

if feature_importance is not None:
    feature_importance.to_csv('results/feature_importance.csv', index=False)

print("💾 Models và kết quả đã được lưu:")
print(f"   • Models: models/{model_type}_*")
print(f"   • Results: results/model_results.json")
print(f"   • Feature importance: results/feature_importance.csv")
print(f"   • Submission: {submission_filename}")

## 11. Tóm tắt và Kết luận

In [ ]:
# Tóm tắt cuối cùng
def final_summary():
    print("="*80)
    print("                    TÓM TẮT DỰ ÁN PREDICTION INTERVALS")
    print("="*80)
    
    print(f"\n🎯 MỤC TIÊU:")
    print(f"   • Xây dựng mô hình dự đoán khoảng giá nhà (90% confidence interval)")
    print(f"   • Tối thiểu hóa Mean Interval Width trên Kaggle")
    print(f"   • Đảm bảo coverage rate ≥ 90%")
    
    print(f"\n📊 PHƯƠNG PHÁP ĐÃ THỰC HIỆN:")
    print(f"   1. Quantile Regression")
    print(f"   2. LightGBM Quantile")
    print(f"   3. Ensemble với Uncertainty Estimation")
    
    print(f"\n🏆 KỂT QUẢ TỐT NHẤT:")
    print(f"   • Phương pháp: {best_method}")
    print(f"   • Mean Interval Width: {cv_results['mean_width']:,.0f} ± {cv_results['std_width']:,.0f}")
    print(f"   • Coverage Rate: {cv_results['mean_coverage']:.3f} ± {cv_results['std_coverage']:.3f}")
    
    print(f"\n📈 SUBMISSION KAGGLE:")
    print(f"   • File: {submission_filename}")
    print(f"   • Samples: {len(submission)}")
    print(f"   • Mean Interval Width: ${(submission['upper'] - submission['lower']).mean():,.0f}")
    
    if feature_importance is not None:
        print(f"\n🔍 TOP 5 FEATURES QUAN TRỌNG:")
        for i, (_, row) in enumerate(feature_importance.head(5).iterrows()):
            print(f"   {i+1}. {row['feature']}: {row['importance']:.3f}")
    
    print(f"\n✅ DELIVERABLES:")
    print(f"   • EDA notebook (Q1)")
    print(f"   • Model training notebook (Q2)")
    print(f"   • Trained models")
    print(f"   • Kaggle submission file")
    print(f"   • Feature importance analysis")
    
    print(f"\n🚀 NEXT STEPS:")
    print(f"   1. Submit {submission_filename} to Kaggle")
    print(f"   2. Monitor leaderboard performance")
    print(f"   3. Iterate and improve based on feedback")
    print(f"   4. Prepare final report and presentation")
    
    print("\n" + "="*80)

final_summary()